In [1]:
import pandas as pd
import numpy as np
import ast
import seaborn
from collections import Counter
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
from bs4 import BeautifulSoup
import logging, gensim, bz2
import os
import re
from operator import itemgetter

In [2]:
import json
import urlparse
from itertools import chain
flatten = chain.from_iterable
from nltk import word_tokenize

In [3]:
def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

### Load Pickled Data

In [4]:
pitt_review = load("pitt_review.pkl")

In [5]:
corpus = load("corpus.pkl")
dictionary = load("dictionary.pkl")

### Start LDA`

In [ ]:
#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word = dictionary, passes=20)
#print(ldamodel.print_topics(topics=50, topn=10))

In [ ]:
ldamodel.save("lda.model")

In [6]:
model =  gensim.models.LdaModel.load('lda.model')

In [7]:
model.print_topics(50,topn=30)

['0.231*pork + 0.082*pull + 0.059*belli + 0.043*meatbal + 0.023*rotat + 0.022*honey + 0.018*parm + 0.014*southern + 0.014*board + 0.013*casbah + 0.012*brule + 0.011*casa + 0.010*mustard + 0.010*stout + 0.010*salumi + 0.010*alfredo + 0.009*bbq + 0.009*beef + 0.009*adana + 0.008*path + 0.008*pricier + 0.008*wagyu + 0.007*chalk + 0.007*satay + 0.007*micro + 0.007*elbow + 0.007*imo + 0.006*sauc + 0.006*beaten + 0.005*fatti',
 '0.120*lunch + 0.030*salad + 0.025*sandwich + 0.020*soup + 0.018*quick + 0.017*stop + 0.016*work + 0.015*fresh + 0.015*day + 0.013*chicken + 0.012*today + 0.012*option + 0.011*healthi + 0.011*downtown + 0.010*special + 0.010*worker + 0.009*order + 0.008*look + 0.008*place + 0.008*menu + 0.007*dinner + 0.007*grab + 0.007*side + 0.007*spot + 0.007*cafe + 0.007*fast + 0.007*great + 0.007*wrap + 0.007*co + 0.006*eat',
 '0.031*food + 0.022*order + 0.021*ask + 0.021*servic + 0.014*never + 0.012*time + 0.012*back + 0.011*manag + 0.011*go + 0.010*said + 0.010*one + 0.010*u + 

### Assigning topics to reviews

In [77]:
topic_dict = []
for name in os.listdir("WordCloudLabels"):
    topic = name
    for filename in os.listdir("WordCloudLabels/" + name):
        topic_id = int(re.search(r'\d+', filename).group())
        sub_topic_re = re.search(r'_\w+', filename)
        if sub_topic_re:
             sub_topic = sub_topic_re.group()[1:]
        else:
            sub_topic=''
        topic_dict.append([topic_id, topic, sub_topic])
    
topic_dict = pd.DataFrame(topic_dict, columns=['Topic_ID', 'Topic', 'Sub_Topic'])        

In [78]:
topic_dict = topic_dict.set_index('Topic_ID').to_dict()

In [79]:
topic_dict['Sub_Topic'][1]

'lunch_work_service'

In [109]:
Topic_list = []
for corp in corpus:
    _srtd = sorted(model[corp], key=itemgetter(1), reverse=1)
    top, score = _srtd[0]
    top_name = topic_dict['Topic'][top] 
    top_level2 = topic_dict['Sub_Topic'][top]
    all_ID = [tup[0] for tup in _srtd]
    all_name = [topic_dict['Topic'][id] for id in All_ID]
    all_level2 = [topic_dict['Sub_Topic'][id] for id in All_ID]
    top_topic_words = [tup[1] for tup in term_list[top]]
    Topic_list.append([top, top_name, top_level2, score, all_ID, all_name, all_level2, _srtd, top_topic_words])

In [110]:
Topic_list =  pd.DataFrame(Topic_list, columns=['Top_ID', 'Top_name', 'Top_level2', 
                                                'Score_top', 'All_ID', 'All_name',
                                                'All_level2', 'All_score', 'Top_Topic_Words'])

In [113]:
pitt_review.columns

Index([u'funny', u'user_id', u'review_id', u'text', u'business_id', u'stars_x',
       u'date', u'useful', u'type_x', u'cool', u'neighborhood', u'hours',
       u'is_open', u'address', u'attributes', u'categories', u'city',
       u'review_count', u'name', u'longitude', u'state', u'stars_y',
       u'latitude', u'postal_code', u'type_y', u'Restaurant', u'clean_text'],
      dtype='object')

In [125]:
reviews = pitt_review[['user_id', 'review_id', 'text', 'name', 'attributes', 'categories']].reset_index()

In [129]:
reviews_labeled = pd.concat([reviews, Topic_list], axis=1)

In [130]:
reviews_labeled.to_csv('reviews_with_topics.csv')

In [131]:
reviews_labeled.to_pickle('reviews_with_topics.pkl')